In [2]:
!pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o Cliente SDK do Gemini
from google import genai
client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
# Instala o framework ADK de agentes do Google
!pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
# importa as bibliotecas do adk
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função para enviar uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
  # Cria um serviço de sessão em memória
  session_service = InMemorySessionService()
  # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
  session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
  # Cria um runner com a sessão e o agente
  runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
  #Cria o conteúdo da mensagem da entrada
  content = types.Content(
      role="user",
      parts=[types.Part(text=message_text)]
  )
  final_response = ""
  #Itera assicronamente eplos eventos retornados durante a execução do agente
  for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
        return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
import google.generativeai as genai
import json
import re


def gerar_enunciado_alternativas_resposta(topico, dificuldade):
    """
    Gera o enunciado de uma questão de física de múltipla escolha,
    as alternativas (A, B, C, D) e a resposta correta usando o Gemini.

    Args:
        topico (str): O tópico da física.
        dificuldade (str): O nível de dificuldade.

    Returns:
        dict: Um dicionário contendo o enunciado, as opções e a resposta correta.
              Retorna None em caso de erro.
    """
    model = genai.GenerativeModel("gemini-2.0-flash")

    prompt = f"""
    Gere uma questão de física de nível {dificuldade} sobre o tópico de {topico} para estudantes do ensino médio.
    A questão deve ser no formato de múltipla escolha com exatamente 4 opções (A, B, C, D).
    Indique claramente a resposta correta ao final da geração.

    Formato de saída desejado:
    {{
      "enunciado": "...",
      "opcoes": ["A) ...", "B) ...", "C) ...", "D) ..."],
      "resposta_correta": "..."
    }}
    """

    try:
        response = model.generate_content(prompt)
        # Remover o bloco ```json e garantir que não há espaços extras no início
        json_string = response.text.strip()
        json_string = re.sub(r'^\s*```json\s*', '', json_string, flags=re.MULTILINE)  # Remove ```json no início
        json_string = re.sub(r'^\s*```\s*', '', json_string, flags=re.MULTILINE) # Remove ``` no final
        questao_info = json.loads(json_string)
        return questao_info
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON da resposta: {e}")
        print(f"Resposta bruta do Gemini: {response.text}")
        return None
    except Exception as e:
        print(f"Erro ao gerar o enunciado, alternativas e resposta: {e}")
        return None

Um carro parte do repouso e acelera uniformemente a uma taxa de 2 m/s² durante 5 segundos. Qual a velocidade final do carro após esse período?
Dados da Questão Gerados:
Enunciado: Um carro parte do repouso e acelera uniformemente a uma taxa de 2 m/s² durante 5 segundos. Qual a velocidade final do carro após esse período?
Opções: ['A) 2.5 m/s', 'B) 5 m/s', 'C) 10 m/s', 'D) 25 m/s']
Resposta Correta: C


In [21]:
def gerar_resolucao(enunciado, resposta_correta):
    """
    Gera a resolução detalhada para a resposta correta de uma questão de física
    usando o Gemini.

    Args:
        enunciado (str): O enunciado da questão.
        resposta_correta (str): A letra da opção correta (ex: "A", "B", "C", "D").

    Returns:
        str: A resolução detalhada da questão em formato de texto com LaTeX.
             Retorna None em caso de erro.
    """
    model = genai.GenerativeModel("gemini-2.0-flash")

    prompt = f"""
    Gere a resolução detalhada e passo a passo para a seguinte questão de física do ensino médio brasileiro:

    {enunciado}

    A resposta correta para esta questão é a opção {resposta_correta}.
    Inclua todos os passos do raciocínio e utilize notação LaTeX para fórmulas e cálculos.
    """

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Erro ao gerar a resolução: {e}")
        return None

# Exemplo de uso da segunda função (depende da saída da primeira):
if __name__ == "__main__":
    questao_data = gerar_enunciado_alternativas_resposta("Ondulatória", "Intermediário")
    print(questao_data["resposta_correta"])
    if questao_data:
        resolucao = gerar_resolucao(questao_data['enunciado'], questao_data['resposta_correta'])
        if resolucao:
            print("\nResolução da Questão:")
            print(resolucao)

Uma onda sonora senoidal de frequência 440 Hz propaga-se no ar com uma velocidade de 340 m/s. Se a frequência da onda for dobrada, mantendo-se o mesmo meio de propagação (ar), o que acontecerá com o comprimento de onda?
C

Resolução da Questão:
## Resolução Detalhada: Comprimento de Onda e Frequência

**Problema:** Uma onda sonora senoidal de frequência 440 Hz propaga-se no ar com uma velocidade de 340 m/s. Se a frequência da onda for dobrada, mantendo-se o mesmo meio de propagação (ar), o que acontecerá com o comprimento de onda?

**Solução:**

1. **Entendimento da Relação Fundamental:**

A relação entre a velocidade da onda ($v$), o comprimento de onda ($\lambda$) e a frequência ($f$) é dada por:

   $v = \lambda \cdot f$

   Onde:

   *  $v$ = velocidade da onda (m/s)
   *  $\lambda$ = comprimento de onda (m)
   *  $f$ = frequência (Hz)

2. **Análise da Situação Inicial:**

Na situação inicial, temos:

   *  $f_1 = 440 \text{ Hz}$
   *  $v = 340 \text{ m/s}$

   Podemos calcular o c

In [23]:
!pip install flask-cors

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import google.generativeai as genai
from google.ai.generativelanguage import ToolConfig, Content
import json
import re

def gerar_enunciado_alternativas_resposta_tool(topico: str, dificuldade: str):
    # ... (seu código da função gerar_enunciado_alternativas_resposta_tool) ...
    pass

def gerar_resolucao_tool(enunciado: str, resposta_correta: str):
    # ... (seu código da função gerar_resolucao_tool) ...
    pass

tools = [
    # ... (sua definição de ferramentas) ...
]
tool_configs = [ToolConfig(name=tool["name"]) for tool in tools]
agent_instructions = """ ... """

def obter_questao_e_resolucao(topico, dificuldade):
    # ... (seu código da função obter_questao_e_resolucao) ...
    pass

app = Flask(__name__)
CORS(app)

@app.route('/gerar_exercicio', methods=['POST'])
def gerar_exercicio():
    data = request.get_json()
    dificuldade = data.get('dificuldade')
    topico = data.get('topico')

    if not dificuldade or not topico:
        return jsonify({'error': 'Dificuldade e tópico são obrigatórios'}), 400

    resultado = obter_questao_e_resolucao(topico, dificuldade)
    return jsonify(resultado)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
